# 第1章: 基本的なプロンプト構造

- [レッスン](#lesson)
- [演習](#exercises)
- [サンプル実験場](#example-playground)

## セットアップ

以下のセットアップセルを実行して、APIキーを読み込み、`get_completion`ヘルパー関数を確立してください。

In [ ]:
# Import python's built-in regular expression library
import re
import boto3
import json

# Import the hints module from the utils package
import os
import sys
module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import hints

# Retrieve the MODEL_NAME variable from the IPython store
%store -r MODEL_NAME
%store -r AWS_REGION

client = boto3.client('bedrock-runtime',region_name=AWS_REGION)

def get_completion(prompt,system=''):
    body = json.dumps(
        {
            "anthropic_version": '',
            "max_tokens": 2000,
            "messages": [{"role": "user", "content": prompt}],
            "temperature": 0.0,
            "top_p": 1,
            "system": system
        }
    )
    response = client.invoke_model(body=body, modelId=MODEL_NAME)
    response_body = json.loads(response.get('body').read())

    return response_body.get('content')[0].get('text')

---

## レッスン

Anthropicでは、従来の[Text Completions API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html)と現在の[Messages API](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-messages.html)の2つのAPIを提供しています。このチュートリアルでは、Messages APIのみを使用します。

Messages APIを使用してClaudeを呼び出すために最低限必要なパラメータは以下の通りです：
- `model`: 呼び出し予定のモデルの[APIモデル名](https://docs.aws.amazon.com/bedrock/latest/userguide/model-ids.html#model-ids-arns)

- `max_tokens`: 停止するまでに生成するトークンの最大数。Claudeはこの最大値に到達する前に停止する場合があります。このパラメータは生成するトークンの絶対最大数のみを指定します。さらに、これは*ハード*停止であり、Claudeが単語や文の途中で生成を停止する場合があります。

- `messages`: 入力メッセージの配列。私たちのモデルは、`user`と`assistant`の会話ターンが交互に行われるように訓練されています。新しい`Message`を作成する際は、messagesパラメータで過去の会話ターンを指定し、モデルが会話の次の`Message`を生成します。
  - 各入力メッセージは`role`と`content`を持つオブジェクトである必要があります。単一の`user`ロールメッセージを指定することも、複数の`user`と`assistant`メッセージを含めることもできます（その場合は交互に配置する必要があります）。最初のメッセージは常にuserロールを使用する必要があります。

また、以下のようなオプションパラメータもあります：
- `system`: システムプロンプト - 詳細は以下で説明します。
  
- `temperature`: Claudeの回答の変動性の度合い。これらのレッスンと演習では、`temperature`を0に設定しています。

全APIパラメータの完全なリストについては、[APIドキュメント](https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-claude.html)をご覧ください。

### 例

正しくフォーマットされたプロンプトに対してClaudeがどのように応答するかを見てみましょう。以下の各セルで、セルを実行し（`shift+enter`）、Claudeの回答がブロックの下に表示されます。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

では今度は、正しいMessages APIフォーマットが含まれていないプロンプトを見てみましょう。これらの不正な形式のプロンプトに対して、Messages APIはエラーを返します。

まず、`messages`配列に`role`と`content`フィールドが不足しているMessages API呼び出しの例です。

> ⚠️ **警告:** プロンプトのmessagesパラメータの不正な形式により、以下のセルはエラーを返します。これは想定される動作です。

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"Hi Claude, how are you?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

こちらは`user`と`assistant`ロール間の交互配置ができていないプロンプトです。

> ⚠️ **警告:** `user`と`assistant`ロールの交互配置の欠如により、Claudeはエラーメッセージを返します。これは想定される動作です。

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

`user`と`assistant`メッセージは**必ず交互に配置する必要があり**、メッセージは**必ず`user`ターンで開始する必要があります**。プロンプトに複数の`user`と`assistant`ペアを含めることも可能です（まるで複数ターンの会話をシミュレートするように）。また、Claudeが中断したところから続けるように、最後の`assistant`メッセージに単語を入力することも可能です（これについては後の章で詳しく説明します）。

#### システムプロンプト

**システムプロンプト**も使用できます。システムプロンプトは、「User」ターンで質問やタスクを提示する前に、**Claudeにコンテキスト、指示、ガイドラインを提供する**方法です。

構造的には、システムプロンプトは`user`と`assistant`メッセージのリストとは別に存在し、したがって別の`system`パラメータに属します（notebookの[セットアップ](#setup)セクションの`get_completion`ヘルパー関数の構造をご覧ください）。

このチュートリアル内で、システムプロンプトを使用する可能性のある箇所では、完了関数に`system`フィールドを提供しています。システムプロンプトを使用したくない場合は、単に`SYSTEM_PROMPT`変数を空の文字列に設定してください。

#### システムプロンプトの例

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))

なぜシステムプロンプトを使用するのでしょうか？ **適切に書かれたシステムプロンプトは、Claudeのパフォーマンスを様々な方法で向上させることができます**。例えば、ルールや指示に従うClaudeの能力を向上させることができます。詳細については、Claudeで[システムプロンプトを使用する方法](https://docs.anthropic.com/claude/docs/how-to-use-system-prompts)に関するドキュメントをご覧ください。

次に演習に進みます。上記の内容を変更せずにレッスンプロンプトを試したい場合は、レッスンnotebookの一番下までスクロールして[**サンプル実験場**](#example-playground)をご覧ください。

---

## 演習
- [演習 1.1 - 3まで数える](#exercise-11---counting-to-three)
- [演習 1.2 - システムプロンプト](#exercise-12---system-prompt)

### 演習 1.1 - 3まで数える
適切な`user` / `assistant`形式を使用して、以下の`PROMPT`を編集してClaudeに**3まで数えさせてください。** 出力では、あなたの解答が正しいかどうかも表示されます。

In [ ]:
# Prompt - this is the only field you should change
PROMPT = "[Replace this text]"

# Get Claude's response
response = get_completion(PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    pattern = re.compile(r'^(?=.*1)(?=.*2)(?=.*3).*$', re.DOTALL)
    return bool(pattern.match(text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_1_hint)

### 演習 1.2 - システムプロンプト

`SYSTEM_PROMPT`を修正して、Claudeが3歳の子供のように応答するようにしてください。

In [ ]:
# System prompt - this is the only field you should change
SYSTEM_PROMPT = "[Replace this text]"

# Prompt
PROMPT = "How big is the sky?"

# Get Claude's response
response = get_completion(PROMPT, SYSTEM_PROMPT)

# Function to grade exercise correctness
def grade_exercise(text):
    return bool(re.search(r"giggles", text) or re.search(r"soo", text))

# Print Claude's response and the corresponding grade
print(response)
print("\n--------------------------- GRADING ---------------------------")
print("This exercise has been correctly solved:", grade_exercise(response))

❓ ヒントが欲しい場合は、以下のセルを実行してください！

In [ ]:
print(hints.exercise_1_2_hint)

### おめでとうございます！

ここまでのすべての演習を解くことができたら、次の章に進む準備ができています。楽しいプロンプト作成を！

---

## サンプル実験場

ここは、このレッスンで示されたプロンプトの例を自由に実験し、プロンプトを調整してClaudeの応答にどのような影響があるかを確認する場所です。

In [ ]:
# Prompt
PROMPT = "Hi Claude, how are you?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "Can you tell me the color of the ocean?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Prompt
PROMPT = "What year was Celine Dion born in?"

# Print Claude's response
print(get_completion(PROMPT))

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [{"Hi Claude, how are you?"}],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

In [ ]:
# Get Claude's response
body = json.dumps(
    {
        "anthropic_version": '',
        "max_tokens": 2000,
        "messages": [
          {"role": "user", "content": "What year was Celine Dion born in?"},
          {"role": "user", "content": "Also, can you tell me some other facts about her?"}
        ],
        "temperature": 0.0,
        "top_p": 1,
        "system": ''
    }
)

response = client.invoke_model(body=body, modelId=MODEL_NAME)

# Print Claude's response
print(response[0].text)

In [ ]:
# System prompt
SYSTEM_PROMPT = "Your answer should always be a series of critical thinking questions that further the conversation (do not provide answers to your questions). Do not actually answer the user question."

# Prompt
PROMPT = "Why is the sky blue?"

# Print Claude's response
print(get_completion(PROMPT, SYSTEM_PROMPT))